In [ ]:
import numpy as np
import os
import math

crystal_coordinates = []
with open("crystal-config", "r") as file:
    for line in file:
        columns = line.strip().split()
        x_coordinate = float(columns[2])
        y_coordinate = float(columns[3])
        z_coordinate = float(columns[4])
        theta = (float(columns[5]))
        crystal_coordinates.append([[x_coordinate, y_coordinate, z_coordinate] , theta])

dd=[]
dd = (crystal_coordinates[0:len(crystal_coordinates)])
float_precision_dtype = np.float64
point_dtype = np.dtype([
    ("x", float_precision_dtype),
    ("y", float_precision_dtype),
    ("z", float_precision_dtype),
])
TOR_dtype = np.dtype([
    ("vx", np.uint),
    ("vy", np.uint),
    ("vz", np.uint),
    ("prob", float_precision_dtype)
])
def safe_division(a, b):
    epsilon = 1e-6
    return np.where(np.abs(b) > epsilon, a / (b + epsilon), np.where(a != 0, np.inf, 0))

def probability(P1, P2, image_matrix_size_mm, voxel_size_mm, voxel_nb):
    voxel_nb = np.rint(np.array(image_matrix_size_mm) / np.array(voxel_size_mm)).astype(np.int32)
    Xplanes = np.linspace(-image_matrix_size_mm[0] / 2, image_matrix_size_mm[0] / 2, voxel_nb[0] + 1)
    Yplanes = np.linspace(-image_matrix_size_mm[1] / 2, image_matrix_size_mm[1] / 2, voxel_nb[1] + 1)
    Zplanes = np.linspace(-image_matrix_size_mm[2] / 2, image_matrix_size_mm[2] / 2, voxel_nb[2] + 1)

    alphaX = safe_division(Xplanes - P1[0], P2[0] - P1[0])
    alphaY = safe_division(Yplanes - P1[1], P2[1] - P1[1])
    alphaZ = safe_division(Zplanes - P1[2], P2[2] - P1[2])

    alphaMin = max(0, min(alphaX[0], alphaX[-1]), min(alphaY[0], alphaY[-1]), min(alphaZ[0], alphaZ[-1]))
    alphaMax = min(1, max(alphaX[0], alphaX[-1]), max(alphaY[0], alphaY[-1]), max(alphaZ[0], alphaZ[-1]))

    if alphaMax <= alphaMin:
        return np.empty((0), dtype=TOR_dtype)

    alpha = np.unique(np.concatenate(([alphaMin], alphaX, alphaY, alphaZ, [alphaMax])))
    alpha = np.sort(alpha[(alpha >= alphaMin) & (alpha <= alphaMax)])

    d12 = np.sqrt(np.sum((P1 - P2) ** 2))
    alphamid = (alpha[:-1] + alpha[1:]) / 2.0
    TOR = np.zeros(alphamid.shape, dtype=TOR_dtype)
    TOR["prob"] = d12 * (alpha[1:] - alpha[:-1]) / np.sum(d12 * (alpha[1:] - alpha[:-1]))
    TOR["vx"] = np.floor((P1[0] + alphamid * (P2[0] - P1[0]) - Xplanes[0]) / voxel_size_mm[0]).astype(np.uint)
    TOR["vy"] = np.floor((P1[1] + alphamid * (P2[1] - P1[1]) - Yplanes[0]) / voxel_size_mm[1]).astype(np.uint)
    TOR["vz"] = np.floor((P1[2] + alphamid * (P2[2] - P1[2]) - Zplanes[0]) / voxel_size_mm[2]).astype(np.uint)

    return TOR

def save_tmp(tmp, iteration):
    for file in os.listdir('.'):
        if file.startswith('tttmp_iteration_') and file.endswith('.npy'):
            os.remove(file)
    filename = f"tttmp_iteration_{iteration}.npy"
    np.save(filename, tmp)

def load_tmp():
    filenames = [file for file in os.listdir('.') if file.startswith('tttmp_iteration_') and file.endswith('.npy')]
    if filenames:
        latest_file = max(filenames)
        return np.load(latest_file)
    else:
        return np.zeros([10, 10, 1000])

tmp = load_tmp()

import numpy as np
input_img = np.zeros([10 , 10 , 1000])
input_img[4:6, 4:6,100:900]= 10

img = np.ones([10 ,10 , 1000])

tmp = np.zeros([10 , 10 ,1000])

import math
import numpy as np
for pp in range(248000 , 249000):
  P1 = np.array(dd[pp][0])
  theta1 = (dd[pp][1])
  #print('pp=' , pp)
  for tt in range(pp+1 , len(dd)):
      P2 = np.array(dd[tt][0])
      theta2 = (dd[tt][1])
      if 178 < np.abs(theta1 - theta2) < 182:
        TOR = probability(P1, P2, image_matrix_size_mm=[10, 10, 1000], voxel_size_mm=[1, 1, 1] , voxel_nb=[10 , 10 , 1000])
        if len(TOR) == 0:
            #print('proj = 0')
            continue
        else:
            #print(proj)
            proj = np.sum(img[TOR["vx"], TOR["vy"], TOR["vz"]] * TOR['prob'])
            if proj ==0:
                continue
            else:
                proj0 = np.sum(input_img[TOR["vx"], TOR["vy"], TOR["vz"]] * TOR['prob'])
                if proj0==0:
                    continue
                else:
                #print('proj0' , proj0)
                    tmp[TOR["vx"], TOR["vy"], TOR["vz"]] +=  (TOR['prob']) * ((proj0) / (proj))
                    save_tmp(tmp, pp)